## Blast Raw Reads againt the NCBI nt database

This code was used to Blast nucleotide sequences from raw reads for sevral Bat CoV's.

A local copy of the nt database was used, which was updated on the 21/01/2021.

Mincoda with python 3.6 was used as an environment, on a Linux workstation.

To reproduce, download the nt database from NCBI,

Make sure blast is on path

in .bashrc add

export PATH="$PATH:~/somefolder/ncbi-blast-2.11.0+/bin"

before running this notebook run:

export BLASTDB=/somepath/database_directory

eg

export BLASTDB=/mnt/255GB_ssd/Data/BLAST/blastdb

Then install required libraries below into conda environment.

In [20]:
import os
import time
import pandas as pd
import numpy as np
from io import StringIO
from Bio import SeqIO
from Bio.Cluster import distancematrix
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import subprocess

In [2]:
DATA_PATH='../../'
FASTA_PATH = DATA_PATH+'fasta/'
COV='RaTG13'
BLAST_OUT=DATA_PATH+COV+'/Blastn/'
MN996532_2='MN996532_2_RaTG13_complete_genome.fa'
SRA_PATH=DATA_PATH+'RaTG13/data/'
Sars_SL3_R1='Sars_SL3_R1_171127.fastq'
Sars_SL3_R2='Sars_SL3_R2_171127.fastq'
DB='nt'

In [5]:
readlist=[]
idlist=[]
f=SRA_PATH+Sars_SL3_R1
start_time=time.time()
for record in SeqIO.parse(f, "fastq"):
    idlist.append(record.id)
    readlist.append(record.seq)
end_time=time.time()
print(f'{Sars_SL3_R1} read took {end_time-start_time} seconds')

Sars_SL3_R1_171127.fastq read took 210.52562999725342 seconds


### Local Blast

In [36]:
#local BLAST
def search(idx, query_string, database_path, outfile):
    start_time=time.time()
    #percentage identity 95%
    per_id = 95
    blastn_cline = NcbiblastnCommandline(
                                         db=database_path, 
                                         evalue=0.001,
                                         outfmt=5, 
                                         perc_identity=per_id,
                                         max_target_seqs=1, 
                                         num_threads=6)
    out, err = blastn_cline(stdin=query_string)
    io_result = StringIO(out)
    blast_records = list(NCBIXML.parse(io_result))
    for blast_record in blast_records:
        if len(blast_record.alignments) == 0:
            continue
        else:
            alignment = blast_record.alignments[0]
            title = alignment.title
            f.write(f"{idx}: {title} \n")
    return 
    

In [38]:
#random reads
num_reads=10000
rand_vals=list(np.random.choice(len(readlist), num_reads))

In [ ]:
hit_data=[]
#open file first and write as we go
outfile=f'{BLAST_OUT+Sars_SL3_R1.split(".")[0]}_{num_reads}_95pct_hits.txt'
f= open(outfile,"a")
for ri in rand_vals:
    hit_data.append(search(ri, str(readlist[ri]), DB, f))
f.close()